## Agile Hardware Design
***
# Combinational Logic

<img src="../resource/logo.svg" alt="agile hardware design logo" style="float:right"/>

## Prof. Scott Beamer
### sbeamer@ucsc.edu

## [CSE 228A](https://classes.soe.ucsc.edu/cse228a/Winter24/)

## Plan for Today

* A Bit of parameterization
* Scala/Chisel conditionals
* _Result:_ comfortably construct combinational circuits

## Loading The Chisel Library Into a Notebook

In [ ]:
interp.load.module(os.Path(s"${System.getProperty("user.dir")}/../resource/chisel_deps.sc"))

In [ ]:
import chisel3._
import chisel3.util._
import chiseltest._
import chiseltest.RawTester.test

## Chisel Multiplexors (Mux)

* Can explicitly instantiate a _mux_
```scala
Mux(select, in1, in0)
```
* _Note:_ input 1 (true case) is first, analogous to ternary (`?`) from Verilog/C
* More flavors of muxes (e.g. `MuxCase`, `Mux1H`) available in [Chisel Library](https://javadoc.io/doc/edu.berkeley.cs/chisel3_2.13/latest/chisel3/util/Mux1H$.html)

<p>
<img src="images/mux.svg" alt="mux schematic" style="width:60%;margin:auto"/>

In [ ]:
class MyMux extends Module {
    val io = IO(new Bundle {
        val s   = Input(Bool())
        val in0 = Input(Bool())
        val in1 = Input(Bool())
        val out = Output(Bool())
    })
    io.out := Mux(io.s, io.in1, io.in0)
}
printVerilog(new MyMux)

## A Bit More on Scala's `class`

* Arguments are constructor parameters
* Make a class instance with `new`, and internals are evaluated on instantiation
* Default scope for internals is public
* Arguments need `val` to be made public
* Will cover methods and overloading later

In [ ]:
class MyClass(argS: String, argI: Int) {
    val name = argS
    println("Created " + argS)
}
val mc = new MyClass("mc", 4)
// mc.name = "foo"
println(mc.name)
// println(mc.argI)

## Parameterizing the Mux Width

* Can use class arguments to parameterize our module
* Recommend using Scala types for parameters, and then casting (if necessary) inside module

<p>
<img src="images/muxw.svg" alt="mux schematic" style="width:60%;margin:auto"/>

In [ ]:
class MyPMux(w: Int) extends Module {
    val io = IO(new Bundle {
        val s   = Input(Bool())
        val in0 = Input(UInt(w.W))
        val in1 = Input(UInt(w.W))
        val out = Output(UInt(w.W))
    })
    io.out := Mux(io.s, io.in1, io.in0)
}
printVerilog(new MyPMux(8))

## Scala `if/else`

* If/else akin to other languages
* _Note:_ due to functional nature of language, if/else returns last value of evaluated clause
* Can omit parenthesis if only one statement for clause
  * If short, style recommends keeping entire if one line

In [ ]:
val condition = true
if (condition) {
    println("true case")
} else {
    println("false case")
}
val x = if (condition) 3 else 4
println(x)

## Contrasting Conditional Execution/Selection

### In Circuit (Chisel Mux)

* Selects based on actual circuit inputs
* Hardware contains both "ways"

```scala
val absX = Mux(x < 0.S, -x, x)
```

<img src="images/absMux.svg" alt="mux schematic" style="width:55%;margin:auto"/>

### During Generation (Scala if/else)

* Executed path generates hardware
* Depends on generator parameters, not circuit inputs

```scala
val invX = if (invert) -x else x
```

<img src="images/invCond.svg" alt="conditional schematic" style="width:55%;margin:auto"/>

## Chisel Tool Flow Frontend (from lecture 2)

<img src="../02-hello/images/frontend.svg" alt="Chisel frontend" style="width:80%;margin:auto"/>

* The generated Circuit (`.fir` file) is a specific design instance, and it can be passed off to a _backend_ for simulation or implementation

## Chisel Execution

### Elaborated Hardware design is a useful byproduct of your Chisel program
* Your Chisel design is a Scala program
* As the program executes, under-the-hood it builds up your design using the Chisel Library
* As the program ends, it _elaborates_ (outputs) the design as a firrtl file (concrete instance)

### Core operations in Chisel are simple, Scala combines them productively
* Chisel _components are simple_ things like logic operators, wires, registers, and modules
* To make a design, these components need to be _instantiated_ and _connected_
* Is _productive_ to use Scala to programmatically instantiate and connect components (meta programming)
* Designing with Chisel is programming _spatially_ (creating & connecting components) instead of _temporally_ (conventional software which is about order of operations)

## Scala Values Are References to Chisel Objects

* Our generators are simply instantiating Chisel objects and connecting them together
  * Scala program allows us to control which objects & connections
* The connect operator (`:=`) assigns output of right hand side to input of left hand side
* Can use Scala references to name intermediate results

In [ ]:
class MyXOR extends Module {
    val io = IO(new Bundle {
        val a   = Input(Bool())
        val b   = Input(Bool())
        val c   = Output(Bool())
    })
    val myGate = io.a ^ io.b
    io.c := myGate
}
printVerilog(new MyXOR)

<img src="images/xorRef.svg" alt="XOR with Scala references" style="width:55%;margin:auto"/>

## Chisel `Wire`

* Sometimes need to connect things, but don't know both ends simultaneously
* Commonly used with `when` construct (next slide)

In [ ]:
class MyXOR2 extends Module {
    val io = IO(new Bundle {
        val a   = Input(Bool())
        val b   = Input(Bool())
        val c   = Output(Bool())
    })
    val myWire = Wire(Bool())
    myWire := io.a ^ io.b
    io.c := myWire
}
printVerilog(new MyXOR2)

<img src="images/xorWire.svg" alt="XOR with Chisel Wire" style="width:55%;margin:auto"/>

## Chisel `when`

* When condition is true, performs Chisel operations contained in block
* Generates selection in hardware
  * Under the hood, Chisel will implement with muxes
* Can use `.otherwise` like else
* Can use `.elsewhen` like if else

In [ ]:
class MyWMux(w: Int) extends Module {
    val io = IO(new Bundle {
        val s   = Input(Bool())
        val in0 = Input(UInt(w.W))
        val in1 = Input(UInt(w.W))
        val out = Output(UInt(w.W))
    })
    when (io.s) {
        io.out := io.in1
    } .otherwise {
        io.out := io.in0
    }
}
printVerilog(new MyWMux(8))

## Chisel's Last Connect Semantics

* Can connect a wire multiple times
* Last connection to be evaluated in program orden "wins"

In [ ]:
class LastC extends Module {
    val io = IO(new Bundle {
        val x   = Input(Bool())
        val y   = Output(UInt())
    })
    val w = Wire(UInt())
    w := 1.U
    when (io.x) {
        w := 7.U
    }
    io.y := w
}
printVerilog(new LastC)

## Last Connect Example

* Absolute Value using `when`

<img src="images/absMux.svg" alt="mux schematic" style="width:55%;margin:auto"/>

In [ ]:
class WhenAbs(w: Int) extends Module {
    val io = IO(new Bundle {
        val x    = Input(SInt(w.W))
        val absX = Output(SInt(w.W))
    })
    io.absX := io.x
    when (io.x < 0.S) {
        io.absX := -io.x
    }
}
printVerilog(new WhenAbs(4))

## Bitwidth Truncation

* Width inference will set widths based on rules for operator
* Operators like add can truncate (`+`,`+%`) or grow (`+&`) to not loose data
* If widths set, will truncate or grow
  * UInts zero extend
  * SInts sign extend
* Consult [Chisel Cheat Sheet](https://github.com/freechipsproject/chisel-cheatsheet/releases/latest/download/chisel_cheatsheet.pdf)

In [ ]:
class MyAdder(w: Int) extends Module {
    val io = IO(new Bundle {
        val a = Input(UInt(w.W))
        val b = Input(UInt(w.W))
        val c = Output(UInt())
    })
    io.c := io.a + io.b
//     io.c := io.a +% io.b
//     io.c := io.a +& io.b
}
printVerilog(new MyAdder(8))

## Example: Sign & Magnitude -> 2's Complement

In [ ]:
class SignMagConv(w: Int) extends Module {
    val io = IO(new Bundle {
        val sign = Input(Bool())
        val mag  = Input(UInt(w.W))
        val twos = Output(UInt((w+1).W))
    })
    when (io.sign) {
        io.twos := ~io.mag +& 1.U
    } .otherwise {
        io.twos := io.mag
    }
}

In [ ]:
printVerilog(new SignMagConv(7))

## Working With Bits

### Read-only access range `x(hi,lo)`
* Access nth bit of x `x(n)`
* _Note:_ can't assign extracted range

### Concatenation
* Combine signals x & y together `Cat(x,y)`

### Fill
* Repeat x, n times `Fill(n,x)`

In [ ]:
class SignExtender(win: Int, wout: Int) extends Module {
    val io = IO(new Bundle {
        val in = Input(UInt(win.W))
        val out = Output(UInt(wout.W))
    })
    assert(win > 0)
    assert(win < wout)
    val signBit = io.in(win-1)
    val extension = Fill(wout-win, signBit)
    io.out := Cat(extension, io.in)
}
printVerilog(new SignExtender(4,8))